In [3]:
! pip install -q kaggle

In [4]:
from google.colab import files

In [ ]:
files.upload()


In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                        title                                        size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------  ------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
themrityunjaypathak/covid-cases-and-deaths-worldwide       Covid Cases and Deaths WorldWide              8KB  2023-02-01 12:22:51           8800        285  1.0              
datascientistanna/customers-dataset                        Shop Customer Data                           23KB  2023-02-07 18:42:21           7152        169  1.0              
amaanansari09/top-100-songs                                Top 100 songs                                 6KB  2023-02-16 18:55:35           2150         72  1.0              
justin2028/unemployment-in-america-per-us-state            Unemployment in America, Per US State       826KB  2023-03-02 07:2

In [ ]:
!kaggle competitions download -c spaceship-titanic

In [ ]:
! mkdir train

In [ ]:
! unzip spaceship-titanic.zip -d train

In [ ]:
import pandas as pd
data = pd.read_csv('/content/train/train.csv')
data

In [ ]:
test = pd.read_csv('/content/train/test.csv')

#Exploring Data

In [ ]:
data.info()

# we have 6 numeric features and 5 categorical features(and aslo PassengerId, Name, and our target Transported).

In [ ]:
data.describe()

# except age all other columns have median = 0 and very high max values which makes those columns very much right skewd with outliers

In [ ]:
data.groupby('Transported')['PassengerId'].count()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
fig, axs = plt.subplots(ncols=2,figsize=(15, 6))



sns.countplot(data=data, x="VIP", hue="Transported",ax=axs[0])
sns.countplot(data=data, x="CryoSleep", hue="Transported",ax=axs[1])


# cryosleep passengers could be transported but many VIP passengers were not transported

# Lets explore "Cabin" feature

In [ ]:
data.Cabin.value_counts()

In [ ]:
df=pd.DataFrame()
df[['Cabin_deck','Cabin_num','Cabin_side']]=data.dropna().Cabin.str.split('/',expand=True)
df=df.astype({'Cabin_num':'int'})

In [ ]:
print(df['Cabin_deck'].value_counts())
print(df['Cabin_side'].value_counts())

In [ ]:
f, ax = plt.subplots(figsize=(11,5))
sns.boxplot(x='Transported', y="Age",  data=data)

# there are outliers in the age column
# transported is same for people between age 20 to 40

In [ ]:
transported = data.loc[data['Transported']==1,"Age"].dropna()
sns.distplot(transported)
plt.title("Transported")

In [ ]:
not_transported = data.loc[data['Transported']==0,"Age"].dropna()
sns.distplot(not_transported)
plt.title("Not Transported")

# analysis of HomePlanet

In [ ]:
data['HomePlanet'].value_counts()

In [ ]:
sns.distplot(a=data[data['HomePlanet']=='Earth']['Transported'],bins=3,kde=False)
plt.title("Earth")
plt.xticks([0,1])
plt.show()
plt.title("Europa")
sns.distplot(a=data[data['HomePlanet']=='Europa']['Transported'],bins=3,kde=False)
plt.xticks([0,1])

plt.show()
plt.title("Mars")
sns.distplot(a=data[data['HomePlanet']=='Mars']['Transported'],bins=3,kde=False)
plt.xticks([0,1])

plt.show()

# many passengers from Earth didnot get transported

In [ ]:
data['Destination'].value_counts()

In [ ]:
sns.distplot(a=data[data['Destination']=='TRAPPIST-1e']['Transported'],bins=3,kde=False)
plt.title("TRAPPIST-1e")
plt.xticks([0,1])
plt.show()
plt.title("55 Cancri e")
sns.distplot(a=data[data['Destination']=='55 Cancri e']['Transported'],bins=3,kde=False)
plt.xticks([0,1])

plt.show()
plt.title("PSO J318.5-22")
sns.distplot(a=data[data['Destination']=='PSO J318.5-22']['Transported'],bins=3,kde=False)
plt.xticks([0,1])

plt.show()

# Missing values treatment

In [ ]:
data.isna().sum()

In [ ]:
fig, axes = plt.subplots(nrows = 2, ncols = 3)    # axes is 2d array (3x3)
axes = axes.flatten()         # Convert axes to 1d array of length 9
fig.set_size_inches(15, 15)


for ax, col in zip(axes, data.describe().columns):
  sns.distplot(data[col], ax = ax)
  ax.set_title(col)

In [ ]:
data[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']] = data[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']].fillna(0)
data['VIP']=data['VIP'].fillna(False)
data['Age']=data['Age'].fillna(data['Age'].median())
data['Destination']=data['Destination'].fillna(data['Destination'].mode()[0])
data['Cabin'] =data['Cabin'].fillna('T/0/S')
data['CryoSleep']=data['CryoSleep'].fillna(False)
data['HomePlanet']=data['HomePlanet'].fillna(data['HomePlanet'].mode()[0])
data.set_index('PassengerId',inplace=True)

test
test[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']] = test[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']].fillna(0)
test['VIP']=test['VIP'].fillna(False)
test['Age']=test['Age'].fillna(test['Age'].median())
test['Destination']=test['Destination'].fillna(test['Destination'].mode()[0])
test['Cabin'] =test['Cabin'].fillna('T/0/S')
test['CryoSleep']=test['CryoSleep'].fillna(False)
test['HomePlanet']=test['HomePlanet'].fillna(test['HomePlanet'].mode()[0])
test.set_index('PassengerId',inplace=True)

#Feature engineering

In [ ]:
data[['Cabin_deck','Cabin_num','Cabin_side']]=data.Cabin.str.split('/',expand=True)
data['Total_spend']=data['RoomService']+data['FoodCourt']+data['ShoppingMall']+data['Spa']+data['VRDeck']

In [ ]:
data=data.drop(['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','Cabin','Name'],axis=1)

In [ ]:
test[['Cabin_deck','Cabin_num','Cabin_side']]=test.Cabin.str.split('/',expand=True)
test['Total_spend']=test['RoomService']+test['FoodCourt']+test['ShoppingMall']+test['Spa']+test['VRDeck']

In [ ]:
test=test.drop(['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','Cabin','Name'],axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder

data[['CryoSleep','VIP','Transported','Cabin_side']]=data[['CryoSleep','VIP','Transported','Cabin_side']].apply(LabelEncoder().fit_transform)
OHE=pd.get_dummies(data[['HomePlanet','Destination','Cabin_deck']])
data_y=data['Transported']
data=pd.concat([data, OHE], axis=1).drop(['HomePlanet','Destination','Cabin_deck','Transported'],axis=1)
data=data.astype({'Cabin_num':'int'})

# ----------------------------------------------------------------------------


test[['CryoSleep','VIP','Cabin_side']]=test[['CryoSleep','VIP','Cabin_side']].apply(LabelEncoder().fit_transform)
OHE=pd.get_dummies(test[['HomePlanet','Destination','Cabin_deck']])
test=pd.concat([test, OHE], axis=1).drop(['HomePlanet','Destination','Cabin_deck'],axis=1)
test=test.astype({'Cabin_num':'int'})

#Models

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(data, data_y, test_size=0.2,random_state=13)

In [ ]:
pip install lazypredict

In [ ]:
from lazypredict.Supervised import LazyClassifier
clf = LazyClassifier(verbose=0,ignore_warnings=True,custom_metric=None, predictions=False)

models, predictions = clf.fit(X_train, X_val, y_train, y_val)
models

# Hyperparameter tuning of LGBM

In [ ]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
lgb_params = {
    
    'n_estimators' :[50,100,125],
    'learning_rate' : [0.03,0.05,0.08],
     'max_depth' : [4,5,6]
}
gbc_cv_model = GridSearchCV(LGBMClassifier(objective='binary'), lgb_params, cv = 5, n_jobs = -1)
gbc_cv_model.fit(X_train, y_train)
print("Best hyperparametres of the model: \n", gbc_cv_model.best_params_)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
LGBM=LGBMClassifier(objective='binary',learning_rate= 0.05, max_depth=5, n_estimators=100)
LGBM.fit(X_train, y_train)
y_pred=LGBM.predict(X_val)
accuracy_score(y_val, y_pred)

# RESULTS ON TEST DATA

In [ ]:
test['Transported']=LGBM.predict(test)

In [ ]:
test.reset_index(inplace = True)

In [ ]:
final_df = test[['PassengerId','Transported']]

In [ ]:
final_df.to_csv("Titanic Spaceship competition Submission.csv")